In [1]:
from macrosim import SeriesAccessor, Causality
import datetime as dt
import pandas as pd

In [2]:
fred = SeriesAccessor(
    key_path='./fred_key.env',
    key_name='FRED_KEY'
)
start_date = dt.datetime(2018, 1, 1)
end_date = dt.datetime(2025, 1, 1)
data = fred.get_series(series_ids=['GDP', 'M2V', 'QBPBSTLKTEQKTBKEQKCOMSTK', 'LRAC64TTUSQ156S'],
                       date_range=(start_date, end_date))

In [7]:
data.causality.is_causal

{'GDP': <Influence.CAUSAL: 'CAUSAL'>,
 'M2V': <Influence.CAUSAL: 'CAUSAL'>,
 'QBPBSTLKTEQKTBKEQKCOMSTK': <Influence.CAUSAL: 'CAUSAL'>,
 'LRAC64TTUSQ156S': <Influence.CAUSAL: 'CAUSAL'>}